In [ ]:
from lstm_train import *


In [ ]:
with open("data.json", "r") as f:
    data = json.load(f)

In [ ]:
print(data.keys())
mfccs = data["mfcc"]
labels = data["labels"]
full_dataset = MfccData(mfccs, labels)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])


In [ ]:
class extract_tensor(nn.Module):
    def forward(self,x):
        # Output shape (batch, features, hidden)
        tensor, _ = x
        # Reshape shape (batch, hidden)
        return tensor.squeeze(0)
    
model = nn.Sequential(
    nn.LSTM(13*259,64,batch_first=True),
    extract_tensor(),
    nn.LSTM(64, 64,batch_first=True),
    extract_tensor(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(32, 10),
    nn.Softmax(),
)

In [ ]:
dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(100):
    running_loss = 0.0
    for step,batch in enumerate(dataloader):
        optimizer.zero_grad()
        loss = loss_function(model(batch[0]), batch[1])
        loss.backward()
        optimizer.step()      
    print(f"EPOCH: {epoch+1} STEP: {step} LOSS: {loss.item()}")


In [ ]:
torch.argmax(model(torch.Tensor(mfccs[600]).reshape(1,13*259)))